# Temperaturas en ciudades de Bolivia
*Una exploración visual con datos de baja resolución de SENAMHI*

In [1]:
# Dependencias

import pandas as pd
import matplotlib.pyplot as plt
import csv
import re
import datetime as dt
from matplotlib import cm, ticker, colors
import numpy as np
from scipy.interpolate import interp1d
from IPython.display import Image, display
import math

In [2]:
# Algunas definiciones de estilo para matplotlib

plt.style.use('resources/clean.mplstyle')

SENAMHI produce datos de temperatura a nivel de ciudad y el INE los consolida y publica mensualmente. Una tabla en excel puede ser descargada [desde el portal del INE](https://www.ine.gob.bo/index.php/medio-ambiente/clima-y-atmosfera/).  

In [3]:
# El código en este cuaderno *debería* funcionar correctamente con nuevas actualizaciones en este documento. 
EXCEL_INE = 'resources/BOLIVIA - TEMPERATURA MEDIA POR CIUDADES, SEGÚN AÑO Y MES, 1990 - 2023.xlsx'

In [4]:
# Definiciones para meses que nos ayudarán a leer meses y producir etiquetas

meses = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril': 4,
        'Mayo': 5,
        'Junio': 6,
        'Julio': 7,
        'Agosto': 8,
        'Septiembre': 9,
        'Octubre': 10,
        'Noviembre': 11,
        'Diciembre': 12
    }

meses_backwards = {meses[m]:m for m in meses.keys()}

Leemos este documento y ponemos la información en una forma cómoda para futuras operaciones. Estos registros son promedios mensuales de mediciones en grados centígrados para ciudades capitales más El Alto.

In [5]:
def parse_ine():
    """
    Selecciona los campos relevantes de `EXCEL_INE` y pone en forma fechas y tipos de valores.
    Es una función particularmente sensible al formato que utiliza el INE, que es súper idiosincrático
    y probablemente cambie en los próximos años (meses?).
    """
    
    excel = pd.read_excel(EXCEL_INE, skiprows=8, skipfooter=5)
    excel = excel.drop(columns=[col for col in excel.columns if 'Unnamed' in col])
    periodo_col = excel.columns[0]
    excel = excel[excel[periodo_col].notna()]
    
    header = ['AÑO'] + list(excel.columns)
    entries = []
    for i, row in excel.iterrows():
        if len(re.findall('[0-9]+', str(row[periodo_col]))) > 0:
            anio = row[periodo_col]
        else:
            entries.append([anio] + list(row))
            
    ine = pd.DataFrame(entries, columns=header)
    ine.columns = [col.strip().lower().replace(' ', '_') for col in ine.columns]
    ine.año = ine.año.astype(str).apply(lambda _: re.findall('[0-9]+', _)[0]).astype(int)
    ine.insert(1, 'mes', ine.periodo.apply(lambda _: meses[_.strip()]))
    ine.drop(columns=['periodo'], inplace=True)
    for col in ine.columns :
        if col not in ['año', 'mes']:
            ine[col] = ine[col].replace('n.d.', None).astype(float)
    ine.insert(0, 'fecha', ine[['año', 'mes']].apply(lambda _: dt.datetime(_['año'], _['mes'], 1), axis=1))

    return ine

ine = parse_ine()
ine

,fecha,año,mes,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
0,1990-01-01,1990,1,15.900000,11.600000,19.196000,11.600000,8.000000,20.800000,25.850000,26.000000,25.450000,8.008000
1,1990-02-01,1990,2,15.050000,11.700000,18.316000,10.950000,7.400000,19.800000,26.400000,27.400000,25.650000,8.307400
2,1990-03-01,1990,3,16.450000,12.100000,19.629000,10.250000,7.800000,21.200000,26.750000,26.900000,26.150000,8.764500
3,1990-04-01,1990,4,16.250000,11.800000,18.215000,9.100000,8.000000,20.400000,25.550000,26.400000,25.950000,8.520000
4,1990-05-01,1990,5,14.100000,10.700000,16.201000,6.050000,5.800000,16.200000,21.450000,23.400000,24.750000,7.114500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2023-06-01,2023,6,11.739931,11.621667,14.671667,4.250000,7.341379,13.943500,21.996667,23.793833,25.345000,5.078333
402,2023-07-01,2023,7,13.288710,12.674194,16.191935,3.980645,5.282258,14.490323,21.851613,25.272581,26.796774,5.080645
403,2023-08-01,2023,8,14.098077,12.277419,17.235484,6.304348,6.374074,16.732258,25.916129,26.404839,28.364516,5.819355
404,2023-09-01,2023,9,16.026786,13.917241,20.548214,10.618750,11.246667,19.934783,27.269643,28.922222,29.683929,8.344643


*Cómo cambia la temperatura en la ciudad a través de los años?*

Primero una vista simple que ayude a comparar meses de distintos años en una misma ciudad. La escala de colores está asociada al orden de años entre 1990 y 2023, con énfasis en 2023. El objetivo es ganar una primera intuición, con unidades fáciles de interpretar, sobre la estacionalidad anual de temperaturas, la dirección de cambios durante este periodo y la particularidad de 2023. 

In [6]:
def linemonths_ine(dep, dataframe=ine, cmap='RdBu_r', fn_prefix='ine_comparacion_mensual_', subtitle_prefix='Temperatura media por mes (°C)', footer='Fuente: SENAMHI via INE', y_pivot=None):
    """
    Produce un gráfico de líneas para una serie de temperaturas en una ciudad.
    Como todo buen código para matplotlib, es innecesariamente verboso y feo para leer.
    Algunos parámetros:
    - dep: el nombre de la ciudad
    - dataframe: la tabla, por defecto `ine`
    - cmap: la escala de colores en el inventario de matplotlib
    - fn_prefix: el prefijo del nombre de file
    - subtitle_prefix: el prefijo del subtítulo
    - footer: el texto al pie de la gráfica
    - y_pivot: la posición en el eje vertical para dibujar una línea de referencia
    """
    
    
    colormap_max = .9
    dfi = dataframe.pivot_table(index='mes', columns='año', values=dep).copy()
    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,colormap_max,len(dfi.columns))]
    highlight = dfi.columns[-1]
    fn = f'plots/{fn_prefix}{dep}.png'

    f, ax = plt.subplots(1,1,figsize=(15,4))

    for año, color in zip(dfi.columns, colormap):

        dfi_y = dfi[año].dropna()
        x_curve = np.linspace(dfi_y.index.min(), dfi_y.index.max(), 1000)
        y_curve = interp1d(x=dfi_y.dropna().index, y=dfi_y.interpolate(), kind=2)(x_curve)

        ax.plot(
            x_curve,
            y_curve, 
            color=color if año != highlight else plt.get_cmap(cmap)(1.), 
            lw=.9 if año != highlight else 1.5, 
            alpha=.5 if año != highlight else .9
        )
        
        ax.scatter(
            dfi.index, 
            dfi[año], 
            color=color if año != highlight else plt.get_cmap(cmap)(1.), 
            s=3, 
            alpha=.9
        )

    ax.grid()
    ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda _, pos: meses_backwards[int(_)] if 12 > _ > 1 else '')
    )
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .03
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(.99, 12.01)
    
    if y_pivot != None:
        ax.axhline(y_pivot, linestyle=':', color='#000', alpha=.8, linewidth=.9)
    
    cbar = f.colorbar(
        cm.ScalarMappable(
            norm=colors.Normalize(dfi.columns[0], highlight), 
            cmap=colors.ListedColormap(plt.get_cmap(cmap)(np.linspace(0,colormap_max,1000))).with_extremes(over=plt.get_cmap(cmap)(1.))
        ),
        ax=ax, 
        pad=.02,
        shrink=.7,
        alpha=.9,
        drawedges=False,
        orientation='vertical',
        ticks=list(range(1995,2025,5)),
        extend='max',
        extendrect=True,
        extendfrac=.08
    )
    cbar.outline.set_alpha(.1)
    
    ax.annotate(
        f'{subtitle_prefix} desde 1990 hasta {meses_backwards[dfi[highlight].dropna().index[-1]]} de {highlight}',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        dep.replace('_',' ').upper(),
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        footer,
        xycoords='axes fraction',
        xy=(.0,-.13),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )

In [7]:
# columnas con nombres de ciudades (por qué `deps` ...¿?)
deps = ine.columns[3:]

In [13]:
# un gráfico para cada ciudad
for dep in deps:
    linemonths_ine(dep)

Es claro que años más recientes registran temperaturas más altas y que 2023, particularmente luego de junio, representa un salto importante. Para comprender mejor cuánto crecen temperaturas sería útil tener un punto de referencia, digamos el promedio mensual de temperaturas entre 1990 y 2020. 

In [8]:
baseline = ine[(ine.fecha > '1990-01-01') & (ine.fecha < '2020-01-01')].groupby('mes')[deps].mean()
baseline

,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
mes,,,,,,,,,,
1,15.587208,13.117519,19.250542,12.365178,9.463570,21.038543,26.768966,26.987987,26.345578,9.205401
2,15.228122,13.115759,19.000237,12.122443,9.307277,20.485316,26.271835,26.751769,26.201299,9.204593
3,15.157043,13.054731,19.122980,11.950590,9.168975,20.095538,26.052419,27.083602,26.352258,8.926920
4,14.783500,12.756556,18.417344,10.212722,8.440978,18.740121,24.618111,26.380111,26.118611,8.104162
5,13.591602,11.905430,16.347544,6.351129,6.448274,15.735818,21.984677,24.273118,25.095806,6.457352
6,12.795460,10.797778,14.503022,4.192853,5.141705,13.854500,20.699000,23.287833,24.416889,5.104747
7,12.336788,10.300753,14.502728,4.221214,4.853763,13.424447,20.539744,22.851884,24.182504,4.861296
8,13.383602,11.221774,16.102734,5.996290,6.271950,15.597275,22.832802,24.436142,25.426495,5.834599
9,14.654222,12.132056,18.027422,8.497944,7.778490,17.594316,24.763389,25.922161,26.640278,7.334297


Construimos una nueva tabla donde cada valor representa la diferencia entre la observación y el promedio en el mismo mes y ciudad durante el periodo de referencia. Esta tabla debería apuntar más fácilmente a dónde, cuándo y en qué dimensión ocurren anomalías en temperaturas.

In [9]:
ine_anom = []
for i, row in ine.iterrows():
    for dep in deps:
        row[dep] = row[dep] - baseline.loc[row['mes'], dep]
    ine_anom.append(row)

ine_anom = pd.DataFrame(ine_anom)
ine_anom

,fecha,año,mes,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
0,1990-01-01,1990,1,0.312792,-1.517519,-0.054542,-0.765178,-1.463570,-0.238543,-0.918966,-0.987987,-0.895578,-1.197401
1,1990-02-01,1990,2,-0.178122,-1.415759,-0.684237,-1.172443,-1.907277,-0.685316,0.128165,0.648231,-0.551299,-0.897193
2,1990-03-01,1990,3,1.292957,-0.954731,0.506020,-1.700590,-1.368975,1.104462,0.697581,-0.183602,-0.202258,-0.162420
3,1990-04-01,1990,4,1.466500,-0.956556,-0.202344,-1.112722,-0.440978,1.659879,0.931889,0.019889,-0.168611,0.415838
4,1990-05-01,1990,5,0.508398,-1.205430,-0.146544,-0.301129,-0.648274,0.464182,-0.534677,-0.873118,-0.345806,0.657148
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2023-06-01,2023,6,-1.055529,0.823889,0.168644,0.057147,2.199674,0.089000,1.297667,0.506000,0.928111,-0.026413
402,2023-07-01,2023,7,0.951922,2.373441,1.689208,-0.240568,0.428495,1.065876,1.311869,2.420696,2.614270,0.219349
403,2023-08-01,2023,8,0.714475,1.055645,1.132749,0.308058,0.102124,1.134983,3.083327,1.968697,2.938021,-0.015244
404,2023-09-01,2023,9,1.372563,1.785186,2.520792,2.120806,3.468176,2.340467,2.506254,3.000061,3.043651,1.010346


Comparamos visualmente estos valores en la misma lógica de los primeros gráficos, donde cada valor representa la diferencia entre la temperatura observada en un mes y ciudad, y la temperatura promedio en el periodo de referencia.

In [11]:
for dep in deps:
    linemonths_ine(
        dep, 
        dataframe=ine_anom, 
        fn_prefix='ine_anomalias_', 
        subtitle_prefix='Anomalías mensuales de temperatura (°C)', 
        footer='Fuente: SENAMHI via INE. Periodo de referencia: 1990 a 2020',
        y_pivot=0
    )

La tendencia a mayores temperaturas en años más recientes es clara en la mayoría de las ciudades, y las temperaturas en 2023, particularmente luego de junio, destacan en todos los casos. *Cómo se comparan anomalías en un mismo año, digamos 2023, entre distintas ciudades?* Realizamos una composición que reuna líneas de todas las ciudades para un mismo año. Asignamos colores a cada ciudad siguiendo el orden de los promedios de anomalías en cada ciudad y año, donde ciudades con mayores anomalías respecto al periodo de referencia aparecen más claras y brillantes, y ciudades con menos diferencias se muestran más opacas.

In [12]:
def plot_compare_anom(año=2023, cmap='magma'):
    """
    Dibuja líneas de anomalías durante un año para todas las ciudades.
    El orden de ciudades según su promedio de anomalías determina atributos de cada línea
    como su color, opacidad y grosor. La etiqueta hace explícito este orden. 
    Mucho código son improvisaciones para manejar situaciones como líneas con campos vacíos o
    la posición de la etiqueta.
    """
    
    def plot_curve(x, y, interp, label=None):
        x_curve = np.linspace(x.min(), x.max(), 1000)
        y_curve = interp1d(x=x, y=y, kind=interp)(x_curve)
        ax.plot(
            x_curve,
            y_curve,
            color=color,
            alpha=alpha,
            lw=(i + 3) ** .4,
            label=label
        )

    dfi = ine_anom.loc[ine_anom.año == año, list(deps) + ['mes']].copy().set_index('mes')
    # order = dfi.iloc[-1].sort_values().index
    order = dfi.mean().sort_values().index

    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,.85,len(order))]
    alphas = [(i + 1) / 10 for i in range(0,len(order))]
    
    f, ax = plt.subplots(1,1,figsize=(len(dfi), 4))

    for i, (dep, color, alpha) in enumerate(zip(order, colormap, alphas)):
        
        labelled = False
        label = ' '.join([i[0].upper() + i[1:] for i in dep.split('_')])
        
        if dfi[dep].isna().sum() > 0:
        
            lines = []
            line = []
            for x, y in dfi[dep].items():
                if not math.isnan(y):
                    line.append({'x':x, 'y':y})
                else:
                    if line:
                        lines.append(line)
                    line = []
            if line:
                lines.append(line)

            for line in lines:

                linedf = pd.DataFrame(line)
                plot_curve(linedf.x, linedf.y, 2 if len(linedf) > 2 else 'linear', None if labelled else label)
                labelled = True
                
        else:
            
            plot_curve(dfi.index, dfi[dep], 2, label)

    ax.grid()
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .1
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(.8,len(dfi) + .2)

    ax.xaxis.set_major_locator(ticker.MaxNLocator(len(dfi)))
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda _, pos: meses_backwards[int(_)] if len(dfi) > _ > 1 else '')
    )

    ax.axhline(0, xmin=.02, xmax=.98, linestyle=':', color='#000', alpha=.8, linewidth=.9)

    legend = ax.legend(
        loc='center right',
        bbox_to_anchor=(1 + (1.35 / len(dfi)), .5),
        labelcolor=list(reversed(colormap)),
        handletextpad=.3,
        reverse=True,
    )

    for label, alpha in zip(legend.get_texts(), reversed(alphas)):
        label.set_alpha(alpha)
        label.set_fontweight('bold')

    for line in legend.get_lines():
        line.set_marker('o')
        line.set_linestyle(' ')
    
    
    ax.annotate(
        'Anomalías mensuales de temperatura (°C) en ciudades',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        año,
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        'Fuente: SENAMHI via INE. Periodo de referencia: 1990 a 2000',
        xycoords='axes fraction',
        xy=(.0,-.13),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    fn = f'plots/ine_anomalias_{año}.png'
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )

Comparemos anomalías mensuales en ciudades entre 2018 y 2023.

In [13]:
for año in reversed(range(2018, 2024)):
    plot_compare_anom(año)

Parece existir un patrón en el orden de ciudades donde observamos mayores temperaturas respecto al periodo de referencia. *Cuán consistente es este patrón desde 1990?* Sería útil comparar anomalías entre ciudades a través de todos los años. Construimos una nueva tabla de anomalías con el mismo periodo de referencia, pero esta vez a escala anual. De esta manera podemos poner de fondo variaciones entre estaciones y observar la tendencia general de temperaturas en los últimos 30 años.

In [14]:
baseline_general = ine[(ine.fecha > '1990-01-01') & (ine.fecha < '2020-01-01')][deps].mean()
baseline_general

sucre         14.642861
la_paz        12.457080
cochabamba    17.974504
oruro          9.267537
potosí         7.974104
tarija        18.220662
santa_cruz    24.452816
trinidad      25.768417
cobija        25.911352
el_alto        7.732062
dtype: float64

In [15]:
ine_anom_anual = ine.groupby('año')[deps].mean().sub(baseline_general)
ine_anom_anual

,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
año,,,,,,,,,,
1990,0.207139,-1.448747,-0.471921,-1.021704,-1.065771,-0.053996,-0.402816,0.023249,-0.765519,-0.273803
1991,0.632139,-1.632080,-0.198588,-1.434204,0.067563,0.042974,-0.148650,-0.551751,-0.590519,-0.178787
1992,0.186306,-1.357080,-0.478588,-1.450870,-1.646831,-0.670662,-0.990316,-0.051751,-0.623852,-0.150920
1993,0.186306,-1.165414,-0.676232,-0.909204,NaN,-0.042885,-0.394483,-1.032054,-0.940519,-0.056928
1994,0.094639,-0.990414,-0.176171,-0.655037,-0.865771,0.171004,0.572184,-0.726751,-0.261352,0.086080
1995,0.248806,-0.448747,-0.491254,-0.530037,0.875896,-0.112329,0.126350,0.339916,0.030315,0.383555
1996,-0.813694,-0.948747,-0.425838,-0.775870,-0.301377,-0.603996,-0.548650,0.064916,-0.719685,-0.359170
1997,-0.522028,-0.357080,-0.425171,-0.834204,-0.140771,0.487671,0.847184,-0.210084,0.134481,-0.473903
1998,0.298806,0.942920,0.492162,0.365796,0.250896,-0.137329,0.322184,-0.060084,-0.961352,0.384605


Y creamos una composición que reune anomalías anuales para cada ciudad donde, como en los gráficos anteriores, los colores son asignados según el promedio de anomalías para cada ciudad, donde ciudades con mayores cambios son representadas con colores más cálidos y brillantes.

In [16]:
def plot_anual_compare(cmap='magma'):
    """
    Esta función es una modificación mínima de la anterior y probablemente 
    debería ser implementada como una extensión con un par de parámetros adicionales.
    """
    
    def plot_curve(x, y, interp, label=None):
        x_curve = np.linspace(x.min(), x.max(), 1000)
        y_curve = interp1d(x=x, y=y, kind=interp)(x_curve)
        ax.plot(
            x_curve,
            y_curve,
            color=color,
            alpha=alpha,
            lw=(i + 3) ** .4,
            label=label
        )

    dfi = ine_anom_anual.copy()
    order = dfi.mean().sort_values().index

    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,.85,len(order))]
    alphas = [(i + 1) / 10 for i in range(0,len(order))]
    
    f, ax = plt.subplots(1,1,figsize=(15, 6))

    for i, (dep, color, alpha) in enumerate(zip(order, colormap, alphas)):
        
        labelled = False
        label = ' '.join([i[0].upper() + i[1:] for i in dep.split('_')])
        
        if dfi[dep].isna().sum() > 0:
        
            lines = []
            line = []
            for x, y in dfi[dep].items():
                if not math.isnan(y):
                    line.append({'x':x, 'y':y})
                else:
                    if line:
                        lines.append(line)
                    line = []
            if line:
                lines.append(line)

            for line in lines:

                linedf = pd.DataFrame(line)
                plot_curve(linedf.x, linedf.y, 2 if len(linedf) > 2 else 'linear', None if labelled else label)
                labelled = True
                
        else:
            
            plot_curve(dfi.index, dfi[dep], 2, label)

    ax.grid()
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .1
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(dfi.index.min() - .2, dfi.index.max() + .2)

    ax.xaxis.set_major_locator(ticker.MaxNLocator(len(dfi)))

    ax.axhline(0, xmin=.02, xmax=.98, linestyle=':', color='#000', alpha=.8, linewidth=.9)

    legend = ax.legend(
        loc='center right',
        bbox_to_anchor=(1 + (3 / len(dfi)), .5),
        labelcolor=list(reversed(colormap)),
        handletextpad=.3,
        reverse=True,
    )

    for label, alpha in zip(legend.get_texts(), reversed(alphas)):
        label.set_alpha(alpha)
        label.set_fontweight('bold')

    for line in legend.get_lines():
        line.set_marker('o')
        line.set_linestyle(' ')
    
    
    ax.annotate(
        'Comparada con el promedio entre 1990 y 2000',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        'Temperatura anual media (°C)',
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        'Fuente: SENAMHI via INE.',
        xycoords='axes fraction',
        xy=(.0,-.1),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    fn = f'plots/ine_anomalias_anuales.png'
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )
    
plot_anual_compare()

El orden de ciudades según sus anomalías de temperatura desde 1990 es similar al observado en los años más recientes.